REsizing

In [2]:
from CellClicker.cell_clicker import ImageViewer
import tkinter as tk
if __name__ == "__main__":
    root = tk.Tk()
    app = ImageViewer(root)
    root.mainloop()


XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\image

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\u1604360\.conda\envs\CellToolsGPU\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "e:\Scott\GitHub\CelFDrive\CellClicker\cell_clicker.py", line 317, in finish_bbox
    self.canvas.coords(self.rect, self.start_x, self.start_y, event.x, event.y)
  File "c:\Users\u1604360\.conda\envs\CellToolsGPU\lib\tkinter\__init__.py", line 2766, in coords
    self.tk.call((self._w, 'coords') + args))]
_tkinter.TclError: wrong # args: should be ".!canvas coords tagOrId ?x y x y ...?"


XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20231110\image

In [1]:
from CellClicker.image_selector_multiphase import load_ui_from_folder

load_ui_from_folder()

2024-04-28 12:41:50.273 python[67278:4377795] +[CATransaction synchronize] called within transaction
2024-04-28 12:41:50.472 python[67278:4377795] +[CATransaction synchronize] called within transaction
2024-04-28 12:41:50.519 python[67278:4377795] +[CATransaction synchronize] called within transaction


Selected XML File: /Users/u1604360/Documents/back up tifs/20240417/user_selections/broken.xml
Selected XML: /Users/u1604360/Documents/back up tifs/20240417/user_selections/broken.xml
/Users/u1604360/Documents/back up tifs/20240417/images/cell_reigons.xml
finding: /Users/u1604360/Documents/back up tifs/20240417/images/20240417_P1_t007.png
E:/Scott/Data/20240417/images/20240417_P1_t007.png
finding: /Users/u1604360/Documents/back up tifs/20240417/images/20240417_P1_t005.png
E:/Scott/Data/20240417/images/20240417_P1_t007.png
E:/Scott/Data/20240417/images/20240417_P1_t005.png
finding: /Users/u1604360/Documents/back up tifs/20240417/images/20240417_P1_t010.png
E:/Scott/Data/20240417/images/20240417_P1_t007.png
E:/Scott/Data/20240417/images/20240417_P1_t005.png
E:/Scott/Data/20240417/images/20240417_P1_t010.png
finding: /Users/u1604360/Documents/back up tifs/20240417/images/20240417_P1_t017.png
E:/Scott/Data/20240417/images/20240417_P1_t007.png
E:/Scott/Data/20240417/images/20240417_P1_t005.p

2024-04-28 12:42:28.984 python[67278:4377795] IMKClient Stall detected, *please Report* your user scenario attaching a spindump (or sysdiagnose) that captures the problem - (imkxpc_bundleIdentifierWithReply:) block performed very slowly (31.36 secs).


In [5]:
correct = "/Users/u1604360/Documents/back up tifs/20240417/images/cell_reigons.xml"
broken = "E:/Scott/Data/20240417/images/20240417_P1_t007.png"
correct.split("cell_reigons.xml")[0]+broken.split("/images/")[1]



'/Users/u1604360/Documents/back up tifs/20240417/images/20240417_P1_t007.png'

In [1]:
from CellClicker.convert_selections_multiphase import convert_selections_multiphase
# imgset = '20240417'
imgset = '20231110'
user = 'Scott'
convert_selections_multiphase('E:/Scott/Data/'+imgset+'/user_selections/Scott.xml', 'E:/Scott/Data/'+imgset+'/images/cell_reigons.xml', 'E:/Scott/Data/'+imgset+'/'+user+'_labels', user)
# convert_selections_multiphase('E:/Scott/Data/20240417/user_selections/Sara.xml', 'E:/Scott/Data/20240417/images/cell_reigons.xml', 'E:/Scott/Data/20240417/new_labels')

{'anaphase': 0}
{'anaphase': 0}
newindex 1
looping {'class_id': 0, 'x_center': 0.13958333333333334, 'y_center': 0.9029166666666667, 'width': 0.03333333333333333, 'height': 0.030833333333333334}
{'anaphase': 0}
{'anaphase': 0}
anaphase
{'prophase': 0, 'earlyprometaphase': 1, 'prometaphase': 2, 'metaphase': 3, 'anaphase': 4}
anaphase
4
E:/Scott/Data/20231110/images/20231110_P01_t001.png
E:/Scott/Data/20231110/images/20231110_P01_t001.png
Image dimensions: height=1200, width=1920
Initial Bounding Box: x_min=236, y_min=1065, x_max=300, y_max=1102
Corrected Bounding Box: x_min=236, y_min=1065, x_max=300, y_max=1102
Contour Bounding Box: x=8, y=5, w=46, h=20
New Bounding Box Coordinates: x_min_new=244, y_min_new=1070, x_max_new=290, y_max_new=1090
46 20
Normalized Output: x_center_new=0.1390625, y_center_new=0.9, width_new=0.023958333333333335, height_new=0.016666666666666666
(4, 0.1390625, 0.9, 0.023958333333333335, 0.016666666666666666)
{'metaphase': 0, 'anaphase': 2}
{'anaphase': 2, 'meta

In [1]:
import os
import cv2
import matplotlib.pyplot as plt

def plot_images_with_boxes(image_folder, label_folder, class_dict):
    # Get list of image files in the folder
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

    for image_file in image_files:
        # Load the image
        image_path = os.path.join(image_folder, image_file)
        image = cv2.imread(image_path)
        
        # Load labels for the current image
        label_path = os.path.join(label_folder, os.path.splitext(image_file)[0] + '.txt')
        
        if not os.path.exists(label_path):
            continue
        
        with open(label_path, 'r') as f:
            labels = f.readlines()

        for label in labels:
            class_id, x_center, y_center, width, height = map(float, label.strip().split())
            class_str = class_dict[int(class_id)]
            
            # Convert YOLO format to top-left and bottom-right coordinates
            x1 = int((x_center - width / 2) * image.shape[1])
            y1 = int((y_center - height / 2) * image.shape[0])
            x2 = int((x_center + width / 2) * image.shape[1])
            y2 = int((y_center + height / 2) * image.shape[0])
            
            # Draw bounding box
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Add label text
            cv2.putText(image, class_str, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Plot the image with bounding boxes
        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

# Example usage:
image_folder = 'E:/Scott/Data/20240417/images'
label_folder = 'E:/Scott/Data/20240417/new_labels'
class_dict = {'prophase': 0, 'earlyprometphase':1, 'prometaphase': 2, 'metaphase': 3, 'anaphase': 4 }
swapped_dict = {v: k for k, v in class_dict.items()}
# commented out so not too large for github
# plot_images_with_boxes(image_folder, label_folder, swapped_dict)


In [20]:


# Test cases
phases = {'prometaphase': 4, 'metaphase': 9, 'anaphase': 15, 'prophase': 0}
sorted_phases = dict(sorted(phases.items(), key=lambda item: item[1], reverse=True))

print(sorted_phases)
# Test when new index is 16 (anaphase)
for new_index in list(range(17))[::-1]:

    print("New index:", new_index)
    print("Phase:", get_phase(new_index, sorted_phases))  # Output: anaphase


{'anaphase': 15, 'metaphase': 9, 'prometaphase': 4, 'prophase': 0}
New index: 16
Phase: anaphase
New index: 15
Phase: anaphase
New index: 14
Phase: metaphase
New index: 13
Phase: metaphase
New index: 12
Phase: metaphase
New index: 11
Phase: metaphase
New index: 10
Phase: metaphase
New index: 9
Phase: metaphase
New index: 8
Phase: prometaphase
New index: 7
Phase: prometaphase
New index: 6
Phase: prometaphase
New index: 5
Phase: prometaphase
New index: 4
Phase: prometaphase
New index: 3
Phase: prophase
New index: 2
Phase: prophase
New index: 1
Phase: prophase
New index: 0
Phase: prophase


[16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [5]:
phase_data

{('E:/Scott/Data/20240417/images/20240417_P1_t047.png',
  '1'): {'prometaphase': 4, 'metaphase': 9, 'anaphase': 15},
 ('E:/Scott/Data/20240417/images/20240417_P1_t014.png', '1'): {},
 ('E:/Scott/Data/20240417/images/20240417_P1_t058.png',
  '1'): {'prometaphase': 4, 'metaphase': 8, 'anaphase': 13},
 ('E:/Scott/Data/20240417/images/20240417_P1_t060.png',
  '1'): {'prometaphase': 5, 'metaphase': 8},
 ('E:/Scott/Data/20240417/images/20240417_P1_t060.png',
  '2'): {'prometaphase': 8, 'metaphase': 16},
 ('E:/Scott/Data/20240417/images/20240417_P1_t054.png',
  '1'): {'prometaphase': 4, 'metaphase': 9, 'anaphase': 14},
 ('E:/Scott/Data/20240417/images/20240417_P2_t025.png', '1'): {}}

In [6]:
labels_data

{('E:/Scott/Data/20240417/images/20240417_P1_t047.png',
  '1'): [{'class_id': 0,
   'x_center': 0.0484375,
   'y_center': 0.9070833333333334,
   'width': 0.028125,
   'height': 0.05416666666666667}, {'class_id': 1,
   'x_center': 0.04739583333333333,
   'y_center': 0.9029166666666667,
   'width': 0.028125,
   'height': 0.05416666666666667}, {'class_id': 2,
   'x_center': 0.04791666666666667,
   'y_center': 0.89875,
   'width': 0.028125,
   'height': 0.05416666666666667}, {'class_id': 3,
   'x_center': 0.0484375,
   'y_center': 0.8945833333333333,
   'width': 0.028125,
   'height': 0.05416666666666667}, {'class_id': 4,
   'x_center': 0.04895833333333333,
   'y_center': 0.8904166666666666,
   'width': 0.028125,
   'height': 0.05416666666666667}, {'class_id': 5,
   'x_center': 0.049479166666666664,
   'y_center': 0.88875,
   'width': 0.028125,
   'height': 0.05416666666666667}, {'class_id': 6,
   'x_center': 0.05,
   'y_center': 0.8854166666666666,
   'width': 0.028125,
   'height': 0.054